In [5]:
# Import libraries
import torch
import pandas as pd
import numpy as np
import subprocess
import os

from utils.helper import extract_metrics_from_output

# 1. Connecting to CUDA Grünau

Run on GPU, because running it on CPU will cost a lot of time (during my experiments, it exeeds 8 hours).


I do not recommend to run it in Google Colab, because it interrupts training process.

If you are not going to use remote servers with multiple GPUs (e. g. Grünau servers), skip this part.

In [6]:
# For gruenau: For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [7]:
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print("Number of available GPUs:", num_gpus)

Number of available GPUs: 3


In [8]:
torch.cuda.get_device_name(1)

'Tesla V100-PCIE-32GB'

In [9]:
# Index of the GPU you want to use (e.g., 0, 1, 2, etc.)
# Choose that one that is not used by other processes
gpu_index = "1"

# 2. Informer

In [ ]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/main_experiments"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
seq_len = "96"
model = "Informer"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning
lr = 0.0001
n_heads = 16
e_layers = 2
d_layers = 1

# List to store the results
informer_results = []

with open(log_file_path, "w") as log_file:

  for i, dataset in enumerate(datasets):
    country = dataset[:2]  # Extract country code (e.g., 'DE', 'GB', etc.)

    log_file.write(f"\n=== Starting experiments for country: {country} ===\n")

    for pred_len in pred_lens:
      model_id = f"{country}_{seq_len}_{pred_len}"

      # Arguments for the command
      command = f"""
      python {script_path} \
        --random_seed 2021 \
        --is_training 1 \
        --root_path "{data_path}" \
        --data_path "{dataset}" \
        --model_id {model_id} \
        --model "{model}" \
        --data "custom" \
        --features M \
        --seq_len {seq_len} \
        --label_len 5 \
        --pred_len {pred_len} \
        --e_layers 2 \
        --d_layers 1 \
        --factor 5 \
        --enc_in "{num_cols[i]}" \
        --dec_in "{num_cols[i]}" \
        --c_out "{num_cols[i]}" \
        --des 'Exp' \
        --train_epochs 10 \
        --patience 3 \
        --n_heads {n_heads} \
        --overlapping_windows \
        --itr 1 --batch_size 32 --learning_rate {lr}
      """
      # Log the country and prediction length
      log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

      # Run the command and capture the output
      process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

      # Capture the output in real-time
      output = []
      for line in process.stdout:
          output.append(line)
          print(line, end='')  # Print in the .ipynb cell
          log_file.write(line)  # Write to the log file

      # Wait for the process to complete
      process.wait()

      # Extract metrics from the captured output
      mse, mae = extract_metrics_from_output(output)

      # Log the extracted metrics
      log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}:\n")
      log_file.write(f"MSE: {mse}, MAE: {mae}\n")

      # Append the results to the informer_results list
      informer_results.append({
          'Country': country,
          'Pred_len': pred_len,
          'MSE': mse,
          'MAE': mae
      })

In [ ]:
# Convert the collected data into DataFrame
informer_df = pd.DataFrame(informer_results)

# Set multi-index 
informer_df.set_index(['Country', 'Pred_len'], inplace=True)
informer_df = informer_df.round(4)

# PatchTST

We separated PatchTST from Informer, because it has additional arguments. It is not so easy to modify f-string (as e. g. distionary) to unpack some arguments with if statement. Moreover, it has different parameter values.

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
seq_len = "96"
model = "PatchTST"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning
lr = 0.0001
n_heads = 16
e_layers = 2
d_layers = 1

# List to store the results
patchtst_results = []

with open(log_file_path, "w") as log_file:

  for i, dataset in enumerate(datasets):
    country = dataset[:2]  # Extract country code (e.g., 'DE', 'GB', etc.)

    log_file.write(f"\n=== Starting experiments for country: {country} ===\n")

    for pred_len in pred_lens:
      model_id = f"{country}_{seq_len}_{pred_len}"

      # Arguments for the command
      command = f"""
      python {script_path} \
        --random_seed 2021 \
        --is_training 1 \
        --root_path "{data_path}" \
        --data_path "{dataset}" \
        --model_id {model_id} \
        --model "{model}" \
        --data "custom" \
        --features M \
        --seq_len {seq_len} \
        --label_len 5 \
        --pred_len {pred_len} \
        --e_layers 2 \
        --d_layers 1 \
        --factor 5 \
        --enc_in "{num_cols[i]}" \
        --dec_in "{num_cols[i]}" \
        --c_out "{num_cols[i]}" \
        --des 'Exp' \
        --train_epochs 10 \
        --patience 3 \
        --n_heads {n_heads} \
        --patch_len 12 \
        --stride 6 \
        --overlapping_windows \
        --itr 1 --batch_size 32 --learning_rate {lr}
      """
      # Log the country and prediction length
      log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

      # Run the command and capture the output
      process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

      # Capture the output in real-time
      output = []
      for line in process.stdout:
          output.append(line)
          print(line, end='')  # Print in the .ipynb cell
          log_file.write(line)  # Write to the log file

      # Wait for the process to complete
      process.wait()

      # Extract metrics from the captured output
      mse, mae = extract_metrics_from_output(output)

      # Log the extracted metrics
      log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}:\n")
      log_file.write(f"MSE: {mse}, MAE: {mae}\n")

      # Append the results to the list
      patchtst_results.append({
          'Country': country,
          'Pred_len': pred_len,
          'MSE': mse,
          'MAE': mae
      })

In [ ]:
# Convert the collected data into DataFrame
patchtst_df = pd.DataFrame(patchtst_results)

# Set multi-index 
patchtst_df.set_index(['Country', 'Pred_len'], inplace=True)
patchtst_df = patchtst_df.round(4)

In [25]:
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 24 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 2

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 28321
val 6577
test 8713
	iters: 100, epoch: 1 | loss: 0.4881925
	speed: 0.0612s/iter; left time: 535.5396s
	iters: 200, epoch: 1 | loss: 0.4415741
	speed: 0.0419s/iter; left time: 362.6325s
	iters: 300, epoch: 1 | loss: 0.4410460
	speed: 0.0412s/iter; left time: 351.9689s
	iters: 400, epoch: 1 | loss: 0.2902627
	speed: 0.0415s/iter; left time: 350.4010s
	iters: 500, epoch: 1 | loss: 0.3088942
	speed: 0.0412s/iter; left time: 343.8548s
	iters: 600, epoch: 1 | loss: 0.3044106
	speed: 0.0414s/iter; left time: 341.2761s
	iters: 700, epoch: 1 | loss: 0.2735647
	speed: 0.0421s/iter; left time: 342.8331s
	iters: 800, epoch: 1 | loss: 0.2792752
	speed: 0.0417s/iter; left time: 335.5625s
Epoch: 1 running time: 0.6292555173238118 min.
Epoch: 1, Steps: 885 | Train Loss: 0.3454897 Vali Loss: 0.2995675 Test Loss: 0.3626922

In [19]:
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  #--d_ff 256 \
  #--d_model 256 \
  --des 'Exp' \
  --itr 2

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 4321
test 8641
	iters: 100, epoch: 1 | loss: 0.7090327
	speed: 0.0694s/iter; left time: 653.2046s
	iters: 200, epoch: 1 | loss: 0.6357286
	speed: 0.0500s/iter; left time: 465.5139s
	iters: 300, epoch: 1 | loss: 0.5953805
	speed: 0.0505s/iter; left time: 464.7455s
	iters: 400, epoch: 1 | loss: 0.4663185
	speed: 0.0504s/iter; left time: 459.0093s
	iters: 500, epoch: 1 | loss: 0.4046238
	speed: 0.0505s/iter; left time: 455.3592s
	iters: 600, epoch: 1 | loss: 0.3477176
	speed: 0.0494s/iter; left time: 440.2458s
	iters: 700, epoch: 1 | loss: 0.4344813
	speed: 0.0500s/iter; left time: 440.9754s
	iters: 800, epoch: 1 | loss: 0.3226214
	speed: 0.0484s/iter; left time: 421.2105s
	iters: 900, epoch: 1 | loss: 0.3889300
	speed: 0.0482s/iter; left time: 415.0947s
Epoch: 1 running time: 0.8037162542343139 min.
Ep

# enc_layers = 2, d_layers = 1

In [5]:
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 6505
test 6457
	iters: 100, epoch: 1 | loss: 0.7141522
	speed: 0.0572s/iter; left time: 538.7338s
	iters: 200, epoch: 1 | loss: 0.6334165
	speed: 0.0384s/iter; left time: 357.9389s
	iters: 300, epoch: 1 | loss: 0.5902387
	speed: 0.0393s/iter; left time: 361.8272s
	iters: 400, epoch: 1 | loss: 0.4588140
	speed: 0.0348s/iter; left time: 316.8726s
	iters: 500, epoch: 1 | loss: 0.4049430
	speed: 0.0388s/iter; left time: 349.9006s
	iters: 600, epoch: 1 | loss: 0.3436919
	speed: 0.0382s/iter; left time: 340.4654s
	iters: 700, epoch: 1 | loss: 0.4383478
	speed: 0.0384s/iter; left time: 338.2228s
	iters: 800, epoch: 1 | loss: 0.3165578
	speed: 0.0383s/iter; left time: 333.2159s
	iters: 900, epoch: 1 | loss: 0.3756514
	speed: 0.0384s/iter; left time: 330.3723s
Epoch: 1 running time: 0.6205641269683838 min.
Epo

# enc_layers = 2, d_layers = 2

In [6]:
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 2 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 6505
test 6457
	iters: 100, epoch: 1 | loss: 0.5454894
	speed: 0.2071s/iter; left time: 1949.4369s
	iters: 200, epoch: 1 | loss: 0.4153013
	speed: 0.1876s/iter; left time: 1746.8217s
	iters: 300, epoch: 1 | loss: 0.3637479
	speed: 0.1845s/iter; left time: 1699.0011s
	iters: 400, epoch: 1 | loss: 0.3130654
	speed: 0.1870s/iter; left time: 1703.5923s
	iters: 500, epoch: 1 | loss: 0.3671271
	speed: 0.1853s/iter; left time: 1669.6942s
	iters: 600, epoch: 1 | loss: 0.3828981
	speed: 0.1905s/iter; left time: 1697.6272s
	iters: 700, epoch: 1 | loss: 0.2786609
	speed: 0.1832s/iter; left time: 1614.5077s
	iters: 800, epoch: 1 | loss: 0.4051742
	speed: 0.1803s/iter; left time: 1570.2727s
	iters: 900, epoch: 1 | loss: 0.3166372
	speed: 0.1838s/iter; left time: 1582.4113s
Epoch: 1 running time: 2.956809914112091 

In [7]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 3 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl3_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 6505
test 6457
	iters: 100, epoch: 1 | loss: 0.5629588
	speed: 0.2633s/iter; left time: 2477.8476s
	iters: 200, epoch: 1 | loss: 0.4396147
	speed: 0.2566s/iter; left time: 2389.3623s
	iters: 300, epoch: 1 | loss: 0.4173067
	speed: 0.2585s/iter; left time: 2380.9574s
	iters: 400, epoch: 1 | loss: 0.3548193
	speed: 0.2377s/iter; left time: 2165.8850s
	iters: 500, epoch: 1 | loss: 0.3554465
	speed: 0.2500s/iter; left time: 2253.1584s
	iters: 600, epoch: 1 | loss: 0.3692197
	speed: 0.2545s/iter; left time: 2268.0729s
	iters: 700, epoch: 1 | loss: 0.3172928
	speed: 0.2442s/iter; left time: 2151.5909s
	iters: 800, epoch: 1 | loss: 0.2884246
	speed: 0.2504s/iter; left time: 2181.0234s
	iters: 900, epoch: 1 | loss: 0.3082856
	speed: 0.2439s/iter; left time: 2100.1119s
Epoch: 1 running time: 3.967679738998413 

In [8]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 4 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl4_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 6505
test 6457
	iters: 100, epoch: 1 | loss: 0.6925252
	speed: 0.3040s/iter; left time: 2860.7458s
	iters: 200, epoch: 1 | loss: 0.5013836
	speed: 0.2943s/iter; left time: 2739.8099s
	iters: 300, epoch: 1 | loss: 0.3475897
	speed: 0.2994s/iter; left time: 2757.7811s
	iters: 400, epoch: 1 | loss: 0.3775092
	speed: 0.2996s/iter; left time: 2730.1056s
	iters: 500, epoch: 1 | loss: 0.4223146
	speed: 0.2955s/iter; left time: 2662.6196s
	iters: 600, epoch: 1 | loss: 0.3139987
	speed: 0.3032s/iter; left time: 2702.1949s
	iters: 700, epoch: 1 | loss: 0.3323351
	speed: 0.2917s/iter; left time: 2570.1443s
	iters: 800, epoch: 1 | loss: 0.2748648
	speed: 0.2911s/iter; left time: 2535.6203s
	iters: 900, epoch: 1 | loss: 0.3644424
	speed: 0.3168s/iter; left time: 2727.8254s
Epoch: 1 running time: 4.775242531299591 

In [9]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl5_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 6505
test 6457
	iters: 100, epoch: 1 | loss: 0.6451275
	speed: 0.3685s/iter; left time: 3467.8304s
	iters: 200, epoch: 1 | loss: 0.4659448
	speed: 0.3537s/iter; left time: 3293.6577s
	iters: 300, epoch: 1 | loss: 0.4373887
	speed: 0.3559s/iter; left time: 3278.6291s
	iters: 400, epoch: 1 | loss: 0.4091117
	speed: 0.3554s/iter; left time: 3237.9175s
	iters: 500, epoch: 1 | loss: 0.4036097
	speed: 0.3617s/iter; left time: 3259.2343s
	iters: 600, epoch: 1 | loss: 0.2723704
	speed: 0.3481s/iter; left time: 3101.9438s
	iters: 700, epoch: 1 | loss: 0.3614541
	speed: 0.3942s/iter; left time: 3472.9975s
	iters: 800, epoch: 1 | loss: 0.3330092
	speed: 0.3308s/iter; left time: 2881.4987s
	iters: 900, epoch: 1 | loss: 0.3676375
	speed: 0.3349s/iter; left time: 2884.1667s
Epoch: 1 running time: 5.624558067321777 

# Fr_data

In [11]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 3 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl3_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.5846890
	speed: 0.0769s/iter; left time: 714.5033s
	iters: 200, epoch: 1 | loss: 0.3475232
	speed: 0.0576s/iter; left time: 529.3475s
	iters: 300, epoch: 1 | loss: 0.3225615
	speed: 0.0546s/iter; left time: 496.3886s
	iters: 400, epoch: 1 | loss: 0.4322594
	speed: 0.0581s/iter; left time: 522.2346s
	iters: 500, epoch: 1 | loss: 0.3482488
	speed: 0.0578s/iter; left time: 513.8622s
	iters: 600, epoch: 1 | loss: 0.3050084
	speed: 0.0547s/iter; left time: 481.0005s
	iters: 700, epoch: 1 | loss: 0.3001569
	speed: 0.0547s/iter; left time: 475.3532s
	iters: 800, epoch: 1 | loss: 0.2735440
	speed: 0.0528s/iter; left time: 453.7545s
	iters: 900, epoch: 1 | loss: 0.3970126
	speed: 0.0536s/iter; left time: 454.9399s
Epoch: 1 running time: 0.8815101385116577 min.
Epo

In [1]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl5_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.6903490
	speed: 0.1461s/iter; left time: 1357.1336s
	iters: 200, epoch: 1 | loss: 0.4752216
	speed: 0.0985s/iter; left time: 905.6919s
	iters: 300, epoch: 1 | loss: 0.3777922
	speed: 0.1132s/iter; left time: 1029.4739s
	iters: 400, epoch: 1 | loss: 0.3623431
	speed: 0.0874s/iter; left time: 785.9209s
	iters: 500, epoch: 1 | loss: 0.3333850
	speed: 0.0939s/iter; left time: 835.3008s
	iters: 600, epoch: 1 | loss: 0.2815011
	speed: 0.1108s/iter; left time: 974.3129s
	iters: 700, epoch: 1 | loss: 0.3888963
	speed: 0.0886s/iter; left time: 770.0692s
	iters: 800, epoch: 1 | loss: 0.3795986
	speed: 0.1013s/iter; left time: 870.1836s
	iters: 900, epoch: 1 | loss: 0.3074558
	speed: 0.1013s/iter; left time: 859.9714s
Epoch: 1 running time: 1.6102526942888895 min.
E

In [10]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 3 \
  --d_layers 3 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el3_dl3_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.4479353
	speed: 0.1135s/iter; left time: 1054.9520s
	iters: 200, epoch: 1 | loss: 0.3741404
	speed: 0.0950s/iter; left time: 873.2347s
	iters: 300, epoch: 1 | loss: 0.3645598
	speed: 0.0867s/iter; left time: 788.3400s
	iters: 400, epoch: 1 | loss: 0.3582897
	speed: 0.0967s/iter; left time: 869.4632s
	iters: 500, epoch: 1 | loss: 0.2958503
	speed: 0.0882s/iter; left time: 784.2466s
	iters: 600, epoch: 1 | loss: 0.2719029
	speed: 0.0906s/iter; left time: 796.9037s
	iters: 700, epoch: 1 | loss: 0.2582170
	speed: 0.0889s/iter; left time: 772.2398s
	iters: 800, epoch: 1 | loss: 0.2941581
	speed: 0.0905s/iter; left time: 777.8923s
	iters: 900, epoch: 1 | loss: 0.2963971
	speed: 0.0938s/iter; left time: 796.6215s
Epoch: 1 running time: 1.4416791081428528 min.
Ep

In [2]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 4 \
  --d_layers 3 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el4_dl3_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.4420350
	speed: 0.1172s/iter; left time: 1088.8234s
	iters: 200, epoch: 1 | loss: 0.2913979
	speed: 0.0851s/iter; left time: 782.4764s
	iters: 300, epoch: 1 | loss: 0.3384283
	speed: 0.0751s/iter; left time: 682.5596s
	iters: 400, epoch: 1 | loss: 0.3608872
	speed: 0.0922s/iter; left time: 828.9005s
	iters: 500, epoch: 1 | loss: 0.3856245
	speed: 0.0812s/iter; left time: 722.2347s
	iters: 600, epoch: 1 | loss: 0.3024383
	speed: 0.0784s/iter; left time: 689.4299s
	iters: 700, epoch: 1 | loss: 0.3283021
	speed: 0.0901s/iter; left time: 783.0923s
	iters: 800, epoch: 1 | loss: 0.2953873
	speed: 0.0834s/iter; left time: 716.6688s
	iters: 900, epoch: 1 | loss: 0.4737537
	speed: 0.0771s/iter; left time: 654.9533s
Epoch: 1 running time: 1.3363105138142903 min.
Ep

In [3]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 4 \
  --d_layers 2 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el4_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.5640328
	speed: 0.0895s/iter; left time: 831.6055s
	iters: 200, epoch: 1 | loss: 0.4044257
	speed: 0.0789s/iter; left time: 725.5688s
	iters: 300, epoch: 1 | loss: 0.2757474
	speed: 0.0623s/iter; left time: 566.6315s
	iters: 400, epoch: 1 | loss: 0.4992739
	speed: 0.0682s/iter; left time: 613.1427s
	iters: 500, epoch: 1 | loss: 0.3228981
	speed: 0.0715s/iter; left time: 635.8237s
	iters: 600, epoch: 1 | loss: 0.2940255
	speed: 0.0767s/iter; left time: 674.4515s
	iters: 700, epoch: 1 | loss: 0.2463365
	speed: 0.0663s/iter; left time: 576.0224s
	iters: 800, epoch: 1 | loss: 0.3332577
	speed: 0.0643s/iter; left time: 552.4396s
	iters: 900, epoch: 1 | loss: 0.3320244
	speed: 0.0716s/iter; left time: 607.7356s
Epoch: 1 running time: 1.1198751091957093 min.
Epo

In [4]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 5 \
  --d_layers 2 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el5_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.2724705
	speed: 0.1007s/iter; left time: 935.3776s
	iters: 200, epoch: 1 | loss: 0.3360111
	speed: 0.0826s/iter; left time: 759.2508s
	iters: 300, epoch: 1 | loss: 0.3031200
	speed: 0.0640s/iter; left time: 581.9662s
	iters: 400, epoch: 1 | loss: 0.3241749
	speed: 0.0747s/iter; left time: 672.0739s
	iters: 500, epoch: 1 | loss: 0.4436989
	speed: 0.0818s/iter; left time: 727.1866s
	iters: 600, epoch: 1 | loss: 0.2832341
	speed: 0.0750s/iter; left time: 658.9216s
	iters: 700, epoch: 1 | loss: 0.2883096
	speed: 0.0759s/iter; left time: 659.3510s
	iters: 800, epoch: 1 | loss: 0.2843474
	speed: 0.0754s/iter; left time: 647.4826s
	iters: 900, epoch: 1 | loss: 0.2853507
	speed: 0.0876s/iter; left time: 743.8912s
Epoch: 1 running time: 1.2261707663536072 min.
Epo

In [ ]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'FR_data.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 4 \
  --d_layers 3 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

In [ ]:
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 24 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

In [3]:
import os
current_path = os.getcwd() + "/datasets/"
dataset = 'df_most_important_columns.csv'

!python -u ./TSLibrary/run.py \
  --task_name long_term_forecast \
  --is_training 1 \
  --root_path $current_path \
  --data_path $dataset \
  --model_id 1 \
  --model "Informer" \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 2 \
  --d_layers 5 \
  --factor 3 \
  --enc_in 3 \
  --dec_in 3 \
  --c_out 3 \
  --des 'Exp' \
  --itr 1

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast_1_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl5_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30433
val 6505
test 6457
	iters: 100, epoch: 1 | loss: 0.6451275
	speed: 0.1165s/iter; left time: 1096.0194s
	iters: 200, epoch: 1 | loss: 0.4659448
	speed: 0.0810s/iter; left time: 754.2153s
	iters: 300, epoch: 1 | loss: 0.4373887
	speed: 0.0835s/iter; left time: 769.0859s
	iters: 400, epoch: 1 | loss: 0.4091117
	speed: 0.0808s/iter; left time: 736.2644s
	iters: 500, epoch: 1 | loss: 0.4036097
	speed: 0.0810s/iter; left time: 730.1276s
	iters: 600, epoch: 1 | loss: 0.2723704
	speed: 0.0809s/iter; left time: 721.2786s
	iters: 700, epoch: 1 | loss: 0.3614541
	speed: 0.0805s/iter; left time: 709.2631s
	iters: 800, epoch: 1 | loss: 0.3330092
	speed: 0.0845s/iter; left time: 735.9239s
	iters: 900, epoch: 1 | loss: 0.3676375
	speed: 0.0790s/iter; left time: 680.1017s
Epoch: 1 running time: 1.315728747844696 min.
Epo

# Original

In [40]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "24"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__24_DE_Informer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30121
val 4321
test 8617
	iters: 100, epoch: 1 | loss: 0.5979588
	speed: 0.1078s/iter; left time: 1003.5939s
	iters: 200, epoch: 1 | loss: 0.3988746
	speed: 0.0981s/iter; left time: 903.5919s
	iters: 300, epoch: 1 | loss: 0.4951838
	speed: 0.0980s/iter; left time: 892.9673s
	iters: 400, epoch: 1 | loss: 0.3251971
	speed: 0.0979s/iter; left time: 882.0710s
	iters: 500, epoch: 1 | loss: 0.2972392
	speed: 0.0979s/iter; left time: 872.1553s
	iters: 600, epoch: 1 | loss: 0.3141234
	speed: 0.0983s/iter; left time: 865.7961s
	iters: 700, epoch: 1 | loss: 0.3582392
	speed: 0.0980s/iter; left time: 853.8956s
	iters: 800, epoch: 1 | loss: 0.3090230
	speed: 0.0979s/iter; left time: 842.9842s
	iters: 900, epoch: 1 | loss: 0.2600375
	speed: 0.0980s/iter; left time: 833.7338s
Epoch: 1 running time: 1.5546314120292664 mi

# A100 80 GB GPU

In [14]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "24"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__24_DE_Informer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30121
val 6481
test 6457
	iters: 100, epoch: 1 | loss: 0.5892869
	speed: 1.7740s/iter; left time: 16517.8096s
	iters: 200, epoch: 1 | loss: 0.3933146
	speed: 1.7834s/iter; left time: 16426.8272s
	iters: 300, epoch: 1 | loss: 0.4908104
	speed: 1.7627s/iter; left time: 16060.3907s
	iters: 400, epoch: 1 | loss: 0.3265559
	speed: 1.7188s/iter; left time: 15488.1032s
	iters: 500, epoch: 1 | loss: 0.2909752
	speed: 1.7761s/iter; left time: 15826.7083s
	iters: 600, epoch: 1 | loss: 0.3200079
	speed: 1.7850s/iter; left time: 15727.7167s
	iters: 700, epoch: 1 | loss: 0.3656467
	speed: 1.7830s/iter; left time: 15531.9936s
	iters: 800, epoch: 1 | loss: 0.3098063
	speed: 1.7816s/iter; left time: 15341.5524s
	iters: 900, epoch: 1 | loss: 0.2568061
	speed: 1.6853s/iter; left time: 14343.6843s
Epoch: 1 running time: 27.6

KeyboardInterrupt: 

# Same but in Grünau

In [ ]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "24"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

# Pred_len 96 with 

In [9]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "96"
model = "Informer"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__96_DE_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.7559140
	speed: 0.1492s/iter; left time: 1386.4834s
	iters: 200, epoch: 1 | loss: 0.6601455
	speed: 0.1277s/iter; left time: 1173.6310s
	iters: 300, epoch: 1 | loss: 0.5010030
	speed: 0.1271s/iter; left time: 1155.7700s
	iters: 400, epoch: 1 | loss: 0.4657636
	speed: 0.1261s/iter; left time: 1133.3460s
	iters: 500, epoch: 1 | loss: 0.4198299
	speed: 0.1291s/iter; left time: 1148.0713s
	iters: 600, epoch: 1 | loss: 0.4128707
	speed: 0.1288s/iter; left time: 1132.1373s
	iters: 700, epoch: 1 | loss: 0.4657623
	speed: 0.1282s/iter; left time: 1114.4254s
	iters: 800, epoch: 1 | loss: 0.4991989
	speed: 0.1250s/iter; left time: 1074.2327s
	iters: 900, epoch: 1 | loss: 0.5064697
	speed: 0.1251s/iter; left time: 1062.3588s
Epoch: 1 running time: 2.01194255749

In [10]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "96"
model = "Informer"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__96_DE_Informer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30049
val 6409
test 6385
	iters: 100, epoch: 1 | loss: 0.7559140
	speed: 0.1697s/iter; left time: 1577.1307s
	iters: 200, epoch: 1 | loss: 0.6601455
	speed: 0.1413s/iter; left time: 1298.6746s
	iters: 300, epoch: 1 | loss: 0.5010030
	speed: 0.1405s/iter; left time: 1277.6612s
	iters: 400, epoch: 1 | loss: 0.4657636
	speed: 0.1374s/iter; left time: 1235.5857s
	iters: 500, epoch: 1 | loss: 0.4198299
	speed: 0.1405s/iter; left time: 1249.6191s
	iters: 600, epoch: 1 | loss: 0.4128707
	speed: 0.1386s/iter; left time: 1218.1083s
	iters: 700, epoch: 1 | loss: 0.4657623
	speed: 0.1401s/iter; left time: 1217.8301s
	iters: 800, epoch: 1 | loss: 0.4991989
	speed: 0.1407s/iter; left time: 1208.3871s
	iters: 900, epoch: 1 | loss: 0.5064697
	speed: 0.1410s/iter; left time: 1197.1356s
Epoch: 1 running time: 2.22249756256

# Seq len 96, pred len 24

In [11]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "24"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "512",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__24_DE_Informer_custom_ftM_sl512_ll48_pl24_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 29705
val 6481
test 6457
	iters: 100, epoch: 1 | loss: 0.7939370
	speed: 0.2490s/iter; left time: 2286.3164s
	iters: 200, epoch: 1 | loss: 0.5406578
	speed: 0.2212s/iter; left time: 2008.8977s
	iters: 300, epoch: 1 | loss: 0.3586418
	speed: 0.2264s/iter; left time: 2033.1200s
	iters: 400, epoch: 1 | loss: 0.2793235
	speed: 0.2209s/iter; left time: 1962.1477s
	iters: 500, epoch: 1 | loss: 0.3177501
	speed: 0.2228s/iter; left time: 1956.5983s
	iters: 600, epoch: 1 | loss: 0.3329939
	speed: 0.2186s/iter; left time: 1897.9783s
	iters: 700, epoch: 1 | loss: 0.3450849
	speed: 0.2251s/iter; left time: 1931.8296s
	iters: 800, epoch: 1 | loss: 0.2893311
	speed: 0.2184s/iter; left time: 1852.6362s
	iters: 900, epoch: 1 | loss: 0.3893343
	speed: 0.2257s/iter; left time: 1891.5038s
Epoch: 1 running time: 3.4559956192

# Seq len 96, pred len 24, but 15/15

In [12]:
import time 
start = time.time()

current_path = os.getcwd() + "/datasets/"
datasets = ['DE_data.csv', 'GB_data.csv', 'ES_data.csv', 'FR_data.csv', 'IT_data.csv']
num_cols = ["5", "5", "3", "3", "3"]
pred_len = "24"
model = "Informer"

for i, dataset in enumerate(datasets):
    model_id = f"_{pred_len}_{dataset[:2]}"  # Create the model_id
    model_arguments = [
                "--task_name", "long_term_forecast",
                "--is_training", "1", #True
                "--root_path", current_path,
                "--data_path", dataset,
                # "--train_epochs", "1",
                "--model_id", model_id,
                "--model", model,
                "--data", "custom", # Use a custom dataloader (same data preparation as in ARIMA)
                "--features", "M", # Multivariate
                "--seq_len", "96",
                "--label_len", "48",
                "--pred_len", pred_len,
                "--e_layers", "2", 
                "--d_layers", "5",
                "--factor", "5",
                "--enc_in", num_cols[i], 
                "--dec_in", num_cols[i], 
                "--c_out", num_cols[i],
                "--des", "Exp",
                "--itr", "2",
            ]

    int_start = time.time()

    model_output = run_output(path_to_run_file, model_arguments)

    #folder_path = f'/content/drive/MyDrive/Masterarbeit/results/{model}/'
    folder_path = f'./results/{model}/'

    # Write model output into txt file
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    result_file_path = os.path.join(folder_path, 'stored_model_output.txt')
    with open(result_file_path, 'a') as f:

        f.write(model_output + "  \n")
        f.write('\n')
        f.write('\n')

    int_end = time.time()
    print(model_output)
    print(f"Time intermediate for {dataset[:2]} dataset:", (int_end - int_start)/60, "min.")


    end = time.time()
print("Total time:", (end - start)/60, "min.")

Use GPU: cuda:0
>>>>>>>start training : long_term_forecast__24_DE_Informer_custom_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl5_df2048_fc5_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 30121
val 6481
test 6457
	iters: 100, epoch: 1 | loss: 0.5979588
	speed: 0.1158s/iter; left time: 1078.4204s
	iters: 200, epoch: 1 | loss: 0.3988746
	speed: 0.0979s/iter; left time: 901.8832s
	iters: 300, epoch: 1 | loss: 0.4951838
	speed: 0.0982s/iter; left time: 894.5374s
	iters: 400, epoch: 1 | loss: 0.3251971
	speed: 0.0973s/iter; left time: 877.1021s
	iters: 500, epoch: 1 | loss: 0.2972392
	speed: 0.0973s/iter; left time: 866.8448s
	iters: 600, epoch: 1 | loss: 0.3141234
	speed: 0.0973s/iter; left time: 857.7271s
	iters: 700, epoch: 1 | loss: 0.3582392
	speed: 0.0970s/iter; left time: 845.2229s
	iters: 800, epoch: 1 | loss: 0.3090230
	speed: 0.0971s/iter; left time: 836.2026s
	iters: 900, epoch: 1 | loss: 0.2600375
	speed: 0.0974s/iter; left time: 829.1754s
Epoch: 1 running time: 1.5416242758433023 mi

# Test

In [19]:
# metrics
np.load("/Users/valentyna/Documents/Master_thesis_new/results/long_term_forecast_1_Informer_custom_ftM_sl10_ll5_pl10_dm512_nh8_el3_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0/metrics.npy")

array([  0.86847895,   1.1777946 ,   1.0852624 ,   3.2318454 ,
       769.5208    ], dtype=float32)

In [21]:
# preds
np.load("/Users/valentyna/Documents/Master_thesis_new/results/long_term_forecast_1_Informer_custom_ftM_sl10_ll5_pl10_dm512_nh8_el3_dl2_df2048_fc3_ebtimeF_dtTrue_Exp_0/pred.npy").shape

(39, 10, 5)